In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
os.chdir('/home/chris/projects/tidal_flat_0d')

In [2]:
from src import features, models
from pyprojroot import here
root = here()
sys.path.append(str(root))
wdir = root / "data" / "interim"

In [3]:
tides = features.load_tide(wdir, "tides-debug.feather")

In [4]:
from src import models

sim = models.Simulation(
    tide_ts=tides,
    land_elev_init=1.15,
    conc_init=0.0,
    conc_bound=1.0,
    grain_diam=3.5e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate=2e-4,
    comp_rate=-3e-3,
    sub_rate=-4e-3,
    slr=0.0,
    solver_method="DOP853",
    runs=1,
    min_depth=0.0015,
    dense_output=True,
    save_inundations=True,
)
# I = sim.tides.find_inundation()

In [8]:
sim.tides.index[]

Inundation @a6a5b0 
index=array([ 5128,  5129,  5130, ..., 14392, 14393, 14394])
tide_elev=array([1.15162168, 1.15174518, 1.15186865, ..., 1.15185932, 1.15173495,
       1.15161055])
land_elev_init=1.1499988292237442
params=Params @5b80d0 
timestep=1.0
conc_init=0.0
conc_bound=1.0
grain_diam=3.5e-05
grain_dens=2650.0
bulk_dens=900
org_rate=0.0002
comp_rate=-0.003
sub_rate=-0.004
slr=0.0
method='DOP853'
min_depth=0.0015
dense_output=True
solver_method='DOP853'
result=None

In [ ]:
I.aggrade()
flood = I.result.attrs["flood"]
index = flood.t
tide_elev = flood.y[0]
conc = flood.y[1]
land_elev = flood.y[2]
depth = tide_elev - land_elev
settle_rate = sim.params.settle_rate

In [ ]:
class IC(pd.DataFrame):

    # normal properties
    _metadata = ["flood"]

    @property
    def _constructor(self):
        return IC

In [ ]:
i = 2
x = np.concatenate((I.result.attrs["flood"].y, I.result.attrs["ebb"].y), axis=1)[i]
y = np.append(I.result.attrs["flood"].y[i], I.result.attrs["ebb"].y[i])
(x == y).all()

In [ ]:
y

In [ ]:
import pandas as pd
class SubDF(pd.DataFrame):

    _metadata = ["pos"]

    def __init__(self, pandas_obj=None, *args, **kwargs):
        super().__init__(pandas_obj, *args, **kwargs)
        self.pos = (1,2,3)

    @property
    def _constructor(*self):
        return SubDF

In [ ]:
x = SubDF(data=df)

In [ ]:
iris.

In [ ]:
z

In [ ]:
class TidalDataFrame(pd.DataFrame):
    
    _metadata = ["params", "pos", "flood", "ebb", "aggradation", "degradation", "start_time", "end_time"]

    @property
    def _constructor(self):
        return TidalDataFrame

class Tides(TidalDataFrame):

    def __init__(self, tide_ts, land_elev_init, params):
        land_elev = land_elev_init - tide_ts.index.values * params.linear_rate_sec
        df = pd.DataFrame(data={"tide_elev": tide_ts.values, "land_elev": land_elev}, index=tide_ts.index.values)
        super().__init__(df)
        self.pos = 0
        self.params = params

In [ ]:
x = Tides(tide_ts=sim.tides.tide_elev, land_elev_init=1.0, params=sim.params)

In [ ]:
1.0 - (tide_ts.index.values * sim.params.linear_rate_sec)

In [ ]:
df = pd.DataFrame(data={"tide_elev": tide_ts.tide_elev.values, "land_elev": tide_ts.land_elev.values}, index=tide_ts.index.values)

In [ ]:
x = TideDataFrame(df)

In [ ]:
x

In [ ]:
class TideDataFrame(pd.DataFrame):

    def __init__(self, **kwargs):
        if all(val is None for val in kwargs.values()):
            columns = [kwarg for kwarg in kwargs.keys() if kwarg != "index"]
            df = pd.DataFrame(columns=columns)
        else:
            data = {key:val for key, val in kwargs.items() if key != "index"}
            df = pd.DataFrame(data=data, index=kwargs['index'])
        super(TideDataFrame, self).__init__(df)
class Tides(TideDataFrame):
    _metadata = ["params", "pos"]

#     @property
#     def _constructor(self):
#         return Tides

    def __init__(self, tide_ts, land_elev_init, params):
        land_elev = land_elev_init - tide_ts.index.values * params.linear_rate_sec
        df = pd.DataFrame()
        super(Tides, self).__init__(
            
        )

    def __repr__(self):
        return "{name} @{id:x} {attrs}".format(
            name=self.__class__.__name__,
            id=id(self) & 0xFFFFFF,
            attrs="".join("\n{}={!r}".format(k, v) for k, v in self.__dict__.items()),
        )

    @property
    def zero_crossings(self):
        crossings = self.loc[
            np.where(
                np.diff(
                    np.signbit(
                        self.tide_elev
                        - (self.land_elev + self.attrs["params"].min_depth)
                    )
                )
            )[0]
            + 1
        ].index.values

        return crossings

    def find_inundation(self):
        remaining_crossings = self.zero_crossings
        while True:
            if len(remaining_crossings) == 0:
                return None
            elif len(remaining_crossings) == 1:
                print("Warning: Partial inundation cycle!")
                start = remaining_crossings[0]
                end = self.index.values[-1]
            elif len(remaining_crossings) > 1:
                start, end = remaining_crossings[0:2]

            if end - start < 4:
                remaining_crossings = remaining_crossings[2:]
                print("Small inundation at t={}->{}. Skipping.")
                continue

            subset = self.loc[start:end]

            assert (
                subset.tide_elev
                > subset.land_elev + self.attrs["params"].linear_rate_sec
            ).all()

            inundation = Inundation(
                index=subset.index.values,
                tide_elev=subset.tide_elev.values,
                land_elev_init=subset.land_elev.values[0],
                params=self.attrs["params"],
            )

            return inundation

In [ ]:
sim.params

In [ ]:
x = Tides(tide_ts=tide_ts, land_elev_init=1.0, params=sim.params)

In [ ]:
tide_ts = sim.tides

In [ ]:
[i==1 for i in kwargs.values()]

In [ ]:
kwarg

In [ ]:
[kwarg for kwarg in kwargs.items() if kwar

In [ ]:
keys, vals = kwargs.keys()

In [ ]:
[i for i in kwargs.keys() if i != "cat"]

In [ ]:
[i for i in kwargs.items()]

In [ ]:
{key:val for key, val in kwargs.items() if key != "cat"}

In [ ]:
kwargs['dog']